In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle
import logging

In [3]:
# Set logging level to INFO
logging.basicConfig(level=logging.INFO)

In [4]:
class Message:
    '''Class to keep sender, timestamp, public (To/Cc) and hidden (Bcc) recipients of a message
    
    Must map the Message class fields in parseMaildir.py for deserialization purposes
    '''
    def __init__(self, From, mtime, To, Cc, Bcc):
        self.From = From
        self.mtime = mtime
        self.To = To
        self.Cc = Cc
        self.Bcc = Bcc

In [5]:
# Parse the pickle files generated by parseMaildir.py
parsedLogsFolder = 'Enron/parsing/'
social_graph = pickle.load(open(parsedLogsFolder + "social.pkl", "rb"))
log = pickle.load(open(parsedLogsFolder + "replay_log.pkl", "rb"))

In [6]:
def prep_static_view(social_graph):
    '''Prepare the `userset` and `chain_head_dict` structs for the static view scenario
    
    In the static view scenario, all users update their encryption key at the beginning of time.
    We then simulate how the updated keys get propagated after replaying all messages in
    chronological order. We instantiate the `userset` and `chain_head_dict` structs as follows:
    
      * We include in `userset` the users that we have full access to their sent messages
      * We boostrap the social graph of each user in the userset with her future recipients
      * We create a `chain_head_dict` struct to keep track of the latest known head of
        a user's friends
    '''
    # Set of users we know the social graph for
    userset = set([])

    # Dictionary to keep track of the latest head known of a user's friends
    chain_head_dict = {}

    # Initialize the latest known head dictionary
    for user in social_graph:
        userset.add(user['from_header'])
        chain_head_dict[user['from_header'], user['from_header']] = 1
        for friend in user['friends']:
            chain_head_dict[(user['from_header'], friend)] = 0

    return userset, chain_head_dict

In [7]:
def eval_head_propagation(chain_head_dict):
    '''Given `chain_head_dict`, count how many entries of the userset users
    are up-to-date, stale, or not updated at all'''
    updated = 0
    stale = 0
    not_updated = 0

    for user, friend in chain_head_dict:
        # If user not in dataset, ignore
        if user not in userset:
            continue
        
        # If this is the entry of the user's own latest head, continue
        if user == friend:
            continue

        # If entry value is 0, then user did not learn of any of her friend's updates
        if chain_head_dict[(user, friend)] == 0:
            not_updated += 1
            continue

        # If the friend is not included in the userset, user knows of the head updated
        # at the beginning of time
        if (friend, friend) not in chain_head_dict:
            updated += 1
        # Else if friend is included in the userset, we check whether the user knows
        # of her friend's latest head, or an older, stale head
        else:
            if chain_head_dict[(user, friend)] == chain_head_dict[(friend, friend)]:
                updated += 1
            else:
                stale += 1

    return updated, stale, not_updated

In [15]:
'''Simulate static view of Autocrypt

* Public and private recipients learn of the sender's latest head
'''
print("Simulating the static view scenario of Autocrypt:")

userset, chain_head_dict = prep_static_view(social_graph)
enc_emails = 0
total_emails = 0

for email in log:
    recipients = email.To | email.Cc | email.Bcc - {email.From}
    
    total_emails += 1
    enc_emails += 1
    
    for recipient in recipients:
        try:
            # If sender does not know of a recipient's head, the email is
            # sent in clear text
            if chain_head_dict[(email.From, recipient)] == 0:
                raise
        except:
            enc_emails -= 1
            break
    
    # For all recipients, update the dict entry for the sender
    for recipient in recipients:
        if email.From in userset:
            latest_head = chain_head_dict[email.From, email.From]
        else:
            latest_head = 1
        chain_head_dict[(recipient, email.From)] = latest_head

updated, stale, not_updated = eval_head_propagation(chain_head_dict)
print("For the friends of the users in the userset, there were %s updates and %s not updated entries." % (updated + stale, not_updated))
print("%s out of the %s emails sent were encrypted." % (enc_emails, total_emails))

Simulating the static view scenario of Autocrypt:
For the friends of the users in the userset, there were 663 updates and 57792 not updated entries.
599 out of the 125761 emails sent were encrypted.


In [9]:
'''Simulate static view of ClaimChain with private claims

* When sending an email to multiple recipients, add capabilities
  for them to read each other's entry in the sender's ClaimChain
* Recipients find out about updates of contacts they have capability to read
  in the sender's chain, even if that contact is not included in the ongoing email thread.
'''
print("Simulating the static view scenario of ClaimChain with private claims:")

userset, chain_head_dict = prep_static_view(social_graph)

for email in log:
    if email.From not in userset:
        continue

    for recipient in email.To | email.Cc | email.Bcc - {email.From}:
        # For all recipients, update the dict entry for the sender
        chain_head_dict[(recipient, email.From)] = chain_head_dict[email.From, email.From]
        logging.debug("User %s updated the head for user %s" % (recipient, email.From))

        # For all recipients, update the dict entries for the other public recipients
        for other in email.To | email.Cc - {recipient}:
            if (email.From, other) in chain_head_dict:
                if not (recipient, other) in chain_head_dict:
                    chain_head_dict[(recipient, other)] = chain_head_dict[(email.From, other)]
                    logging.debug("User %s updated the head for user %s" % (recipient, other))

updated, stale, not_updated = eval_head_propagation(chain_head_dict)
print("There were %s updates on the latest head dictionary, but %s entries are not updated."
      % (updated + stale, not_updated))

Simulating the static view scenario of ClaimChain with private claims:
There were 475 updates on the latest head dictionary, but 60804 entries are not updated.
